NOTE:
-----
다음의 셀의 명령을 실행하고 넘어가자. 곧 필요하게 된다.

In [3]:
%load_ext sql
%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

In [4]:
%%sql
pragma foreign_keys = ON; -- WARNING: by default off in sqlite
drop table if exists product; -- This needs to be dropped if exists, see why further down!
drop table if exists company;
create table company (
    cname varchar primary key, -- company name uniquely identifies the company.
    stockprice money, -- stock price is in money 
    country varchar); -- country is just a string
insert into company values ('GizmoWorks', 25.0, 'USA');
insert into company values ('Canon', 65.0, 'Japan');
insert into company values ('Hitachi', 15.0, 'Japan');
create table product(
       pname varchar primary key, -- name of the product
       price money, -- price of the product
       category varchar, -- category
       manufacturer varchar, -- manufacturer
       foreign key (manufacturer) references company(cname));
insert into product values('Gizmo', 19.99, 'Gadgets', 'GizmoWorks');
insert into product values('SingleTouch', 149.99, 'Photography', 'Canon');
insert into product values('PowerGizmo', 29.99, 'Gadgets', 'GizmoWorks');
insert into product values('MultiTouch', 203.99, 'Household', 'Hitachi');

 * sqlite://
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [5]:
%%sql
DROP TABLE IF EXISTS franchise;
CREATE TABLE franchise (name TEXT, db_type TEXT);
INSERT INTO franchise VALUES ('Bobs Bagels', 'NoSQL');
INSERT INTO franchise VALUES ('eBagel', 'NoSQL');
INSERT INTO franchise VALUES ('BAGEL CORP', 'MySQL');

DROP TABLE IF EXISTS store;
CREATE TABLE store (franchise TEXT, location TEXT);
INSERT INTO store VALUES ('Bobs Bagels', 'NYC');
INSERT INTO store VALUES ('eBagel', 'PA');
INSERT INTO store VALUES ('BAGEL CORP', 'Chicago');
INSERT INTO store VALUES ('BAGEL CORP', 'NYC');
INSERT INTO store VALUES ('BAGEL CORP', 'PA');

DROP TABLE IF EXISTS bagel;
CREATE TABLE bagel (name TEXT, price MONEY, made_by TEXT);
INSERT INTO bagel VALUES ('Plain with shmear', 1.99, 'Bobs Bagels');
INSERT INTO bagel VALUES ('Egg with shmear', 2.39, 'Bobs Bagels');
INSERT INTO bagel VALUES ('eBagel Drinkable Bagel', 27.99, 'eBagel');
INSERT INTO bagel VALUES ('eBagel Expansion Pack', 1.99, 'eBagel');
INSERT INTO bagel VALUES ('Plain with shmear', 0.99, 'BAGEL CORP');
INSERT INTO bagel VALUES ('Organic Flax-seed bagel chips', 0.99, 'BAGEL CORP');

DROP TABLE IF EXISTS purchase;
-- Note that date is an int here just to simplify things
CREATE TABLE purchase (bagel_name TEXT, franchise TEXT, date INT, quantity INT, purchaser_age INT);
INSERT INTO purchase VALUES ('Plain with shmear', 'Bobs Bagels', 1, 12, 28);
INSERT INTO purchase VALUES ('Egg with shmear', 'Bobs Bagels', 2, 6, 47);
INSERT INTO purchase VALUES ('Plain with shmear', 'BAGEL CORP', 2, 12, 24);
INSERT INTO purchase VALUES ('Plain with shmear', 'BAGEL CORP', 3, 1, 17);
INSERT INTO purchase VALUES ('eBagel Expansion Pack', 'eBagel', 1, 137, 5);
INSERT INTO purchase VALUES ('Plain with shmear', 'Bobs Bagels', 4, 24, NULL);

 * sqlite://
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

다시 한 번 Order By
================
* SQL-89 의 명세에는 다음과 같은 명령을 금지하고 있다. 그렇지만, 무엇을 하려는 질의문인지는 이해할 수 있다.
> SELECT pname FROM Product ORDER BY Price

좀 더 비직관적인 ORDER BY 질의문을 살펴보자.

In [4]:
%sql SELECT * FROM Product;

Done.


pname,price,category,manufacturer
Gizmo,19.99,Gadgets,GizmoWorks
SingleTouch,149.99,Photography,Canon
PowerGizmo,29.99,Gadgets,GizmoWorks
MultiTouch,203.99,Household,Hitachi


In [5]:
%%sql SELECT pname,price FROM Product
ORDER BY pname

Done.


pname,price
Gizmo,19.99
MultiTouch,203.99
PowerGizmo,29.99
SingleTouch,149.99


In [6]:
%%sql SELECT pname FROM Product
ORDER BY Price

Done.


pname
Gizmo
PowerGizmo
SingleTouch
MultiTouch


In [7]:
%%sql SELECT distinct pname FROM Product
ORDER BY Price

Done.


pname
Gizmo
PowerGizmo
SingleTouch
MultiTouch


* Postgres 와 같은 데이터베이스는 이런 문장을 허용하지 않는다. 하지만, SQLite과 같은 경우 처리를 한다. 어째서 그럴까?

Set 연산자
=======
다음과 같이 세 개의 테이블을 생성하자:
* R is {1,2,3,4,5}
* S is {}
* T is {1,4,7,10}


In [8]:
# Create tables & insert some random numbers
# Note: in Postgresql, try the generate_series function...
%sql DROP TABLE IF EXISTS R; DROP TABLE IF EXISTS S; DROP TABLE IF EXISTS T;
%sql CREATE TABLE R (A int); CREATE TABLE S (A int); CREATE TABLE T (A int);
for i in range(1,6):
    %sql INSERT INTO R VALUES (:i)
for i in range(1,11,3):
    %sql INSERT INTO T VALUES (:i)

Done.
Done.
Done.
Done.
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


$R \cap (S \cup T) = \{1,4\}$ 의 결과를 얻도록 해보자.

In [9]:
%%sql SELECT DISTINCT R.A FROM R, S, T
WHERE R.A=S.A OR R.A=T.A

Done.


A


**결과가 왜 비었을까?**

*이 질의문의 연산 순서를 다시 살펴보자:*
1. `R,S,T`의 벡터곱 
2. `WHERE` 절을 사용하여 중간 테이블을 필터처리한다.

(1)의 결과를 보자:

In [10]:
%sql SELECT DISTINCT R.A FROM R, S, T;

Done.


A


벡터곱이 공집합이다.`S` 이 비었기 때문이다!

Union
-----

`UNION` 이라는 새로운 명령을 배워보자:

In [11]:
%%sql
SELECT R.A FROM R, S WHERE R.A=S.A
UNION -- this is an explicit keyword!
SELECT R.A FROM R, T WHERE R.A=T.A

Done.


A
1
4


* 중복된 값이 없다 (union 은 집합이다)
* 중복을 원한다면 UNION ALL을 사용한다
  * R = {1,2,3,4,5}
  * S = {1,2,3,4,5}
  * T = {1,4,7,10}

In [12]:
%sql DROP TABLE IF EXISTS S; CREATE TABLE S (A int);
for i in range(1,6):
    %sql INSERT INTO S VALUES (:i)

Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


In [13]:
%%sql
-- UNION ALL example- notice that 1 and 4 occur twice!
SELECT R.A FROM R, S WHERE R.A=S.A
UNION ALL
SELECT R.A FROM R, T WHERE R.A=T.A

Done.


A
1
2
3
4
5
1
4


다른 집합 연산자: INTERSECT, EXCEPT
-------------------------------------

아래의 예제를 살펴보자

In [14]:
%%sql
SELECT R.A FROM R, S, T WHERE R.A = S.A
INTERSECT
SELECT R.A FROM R, S, T WHERE R.A = T.A

Done.


A
1
4


In [15]:
%%sql
SELECT R.A FROM R, S, T WHERE R.A = S.A
EXCEPT
SELECT R.A FROM R, S, T WHERE R.A = T.A

Done.


A
2
3
5


이 노트북에서 계속 사용될 예제: BAGELS
===========================

이 주제의 목차:
* 중첩 질의문: 동기와 예제
* Aggregation: 개요, group-by 용법
* Null 값, Outer vs. Inner Joins

이야기:
------
* *eBagel* 은 핫 이슈인 NoSQL 기반 베이글 기업 공간의 새로운 기술 지향 스타트업이다.
* eBagel은 $1억의 벤처캐피털 자금을 지원 받았다. 매출 실적이 줄어들고 있는 지금, 당신이 이 기업에 데이터를 분석하여 왜 그런지 알아보라는 임무를 받았다.

다음의 테이블의 값을 삽입하는 것으로 시작해보자(위에서 생성하였음):
> Franchise(name TEXT, db_type TEXT)

> Store(franchise TEXT, location TEXT)

> Bagel(name TEXT, price MONEY, made_by TEXT)

> Purchase(bagel_name TEXT, franchise TEXT, date INT, quantity INT, purchaser_age INT)

데이터를 불러들여 보자.

Union
-----

먼저, PA **또는** NYC에 있는 franchises들을 조사해서 경쟁자들이 누군지 알아보자:

In [7]:
%%sql 
SELECT franchise FROM store WHERE location = 'NYC'
UNION
SELECT franchise FROM store WHERE location = 'PA';

 * sqlite://
Done.


franchise
BAGEL CORP
Bobs Bagels
eBagel


Intersect: 미묘한 문제...
--------------------------

eBagel의 CEO는 여러 지역으로 성공적으로 확장한 베이글 회사들의 기술이 무엇인지 알고 싶어 한다. `INTERSECT` 연산자를 사용하여 PA **와** NYC 소재 franchises의 종류를 알아 보자:

In [17]:
%%sql
SELECT f.db_type
FROM franchise f, store s 
WHERE f.name = s.franchise AND s.location = 'NYC'
INTERSECT
SELECT f.db_type
FROM franchise f, store s 
WHERE f.name = s.franchise AND s.location = 'PA'

Done.


db_type
MySQL
NoSQL


*무슨 일이 일어났나?*

데이터를 보면, "MySQL"만 결과로 얻어야 한다:

In [8]:
%%sql 
SELECT f.name, s.location, f.db_type
FROM franchise f, store s 
WHERE f.name = s.franchise;

 * sqlite://
Done.


name,location,db_type
Bobs Bagels,NYC,NoSQL
eBagel,PA,NoSQL
BAGEL CORP,Chicago,MySQL
BAGEL CORP,NYC,MySQL
BAGEL CORP,PA,MySQL


무슨 일이 일어난 것인가? 왜 이런 일이 일어나는지 질의문의 연산들을 나눠보면 알 수 있다:

In [19]:
%%sql SELECT f.db_type 
FROM franchise f, store s 
WHERE f.name = s.franchise AND s.location = 'NYC'

Done.


db_type
NoSQL
MySQL


In [20]:
%%sql SELECT f.db_type
FROM franchise f, store s
WHERE f.name = s.franchise AND s.location = 'PA'

Done.


db_type
NoSQL
MySQL


근본적으로 우리가 `INTERSECT` 연산을 우리가 실제 원하는 속성들이 아닌, 결과로 얻은 속성 값에 적용하였기 때문에 이런 문제가 일어났다.

중첩 질의문
========

위의 문제를 해결하는 한 방법으로 *중첩 질의문*을 알아보자. 중첩 질의문이란 질의문 내의 질의문으로, 안의 질의문이 내 놓은 결과를 밖의 질의문이 활용하는 연산을 말한다.

In [21]:
%%sql
SELECT f.db_type
FROM franchise f
WHERE f.name IN (
    SELECT s.franchise FROM store s WHERE s.location = 'NYC')
  AND f.name IN (
    SELECT s.franchise FROM store s WHERE s.location = 'PA');

Done.


db_type
MySQL


또 다른 중첩 질의문: eBagel의 CEO가 베이글 회사들이 지원하는 데이터베이스들 중 20 대 사람들이 많이 쓰는 데이터베이스는 무엇인지 궁금해 한다. 

In [22]:
%%sql
SELECT f.db_type
FROM franchise f
WHERE f.name IN (
    SELECT b.made_by
    FROM bagel b, purchase p
    WHERE b.name = p.bagel_name 
      AND p.purchaser_age >= 20 AND p.purchaser_age < 30);

Done.


db_type
NoSQL
MySQL


중첩 질의문은 무척 강력하고 편리하다. 위의 결과를 얻기 위해 중첩 질의문을 쓰지 않고도 가능할까? 같은 결과를 돌려줄가?

In [23]:
%%sql
SELECT f.db_type
FROM franchise f, bagel b, purchase p
WHERE f.name = b.made_by 
  AND b.name = p.bagel_name 
  AND p.purchaser_age >= 20 AND p.purchaser_age < 30;

Done.


db_type
MySQL
NoSQL
MySQL
NoSQL


**중복 결과에 유의하자!** `DISTINCT` 추가하여 동치의 결과를 얻을 수 있다. 한 번 시도해보자. 

다음의 키워드들 역시 중첩 질의문의 결과에 쓸 수 있다:
* `ALL`
* `ANY`
* `EXISTS`

**불행하게도, `ALL` 과 `ANY` 는 우리가 사용 중인 SQLite에서는 지원되지 않는다 - 예제는 수업 자료 참고.**  `EXISTS`의 예는 살펴볼 수 있다. 다음 질문에 답한다고 해보자. 경쟁사보다 더 저럼하게 파는 제품이 존재하는가?

In [9]:
%%sql
SELECT b.name, b.price
FROM bagel b
WHERE b.made_by = 'eBagel'
  AND EXISTS (SELECT name 
              FROM bagel 
              WHERE made_by <> 'eBagel' 
                    AND price > b.price);

 * sqlite://
Done.


name,price
eBagel Expansion Pack,1.99


안 쪽의 중첩 질의의 결과가 밖의 메인 질의에서 활용이 되기 때문에 *corrleated* 질의가 된다. 변수의 활용 범위도 살펴볼 필요가 있다. 이 중첩 질의문을 하나의 SFW 질의로 바꿀 수도 있다. 한 번 생각해보자.

여기까지 요약 정리:
--------------

SQL:
* 데이터를 조작하는 상위 계층의 선언적 언어이다(DML, Data Manipulation Language)
* 모든 일은 SFW 블럭에서 일어난다
* 집합 연산자는 강력하지만 고려해야 할 미묘한 부분이 있다!
* 강력한 중첩 질의문을 사용할 수 있다
    * **SQLite보다 더 많은 기능을 제공하는 다른 DBMS를 쓸 때 그 진가를 알 수 있다!**

Aggregation
=======

SQL 은 집계를 위한 여러 연산들을 제공한다:
* `SUM`
* `COUNT`
* `AVG`
* `MIN`
* `MAX`

*`COUNT`* 를 제외하고는 모든 집게 연산자들은 하나의 속성에만 적용할 수 있다

간단한 예제를 살펴보자:

In [25]:
%sql SELECT AVG(price) FROM bagel WHERE made_by = 'eBagel';

Done.


AVG(price)
14.99


출력되는 스키마의 속성 이름을 재정의할 수 있다. 집계시 유용하다.

In [26]:
%sql SELECT COUNT(*) AS "Number of Stores in PA" FROM store WHERE location = 'PA';

Done.


Number of Stores in PA
2


만약 베이글 사회가 몇 개의 지역으로 이루어졌는지 알아보고자 한다면?

In [27]:
%sql SELECT COUNT(location) FROM store;

Done.


COUNT(location)
5


실제로 우리가 원하는 결과는 중복 값을 제외한 집계 값이다.

In [28]:
%sql SELECT COUNT(DISTINCT location) FROM store;

Done.


COUNT(DISTINCT location)
3


산업 매출 데이터를 구했다고 해보자. 그리고 그 데이터를 이용해 베이글 산업의 규모를 알아보고 싶다. 이 문제의 답을 알기 위해 *SQL*문을 어떻게 작성하면 될까?

In [29]:
%%sql
SELECT SUM(b.price * p.quantity) AS net_sales
FROM bagel b, purchase p
WHERE b.name = p.bagel_name;

Done.


net_sales
432.99


시장에서 기회가 상당히 크다! eBagel의 CEO는 이 소식에 매우 기뻤다. 그리고 좀 더 세부적인 정보를 얻기 원한다. 회사 별 수익 규모가 어떻게 되는지 알아보고 싶다. 다행스럽게도, **군집과 집계는 같이 쓸 수 있다**

In [30]:
%%sql
SELECT b.made_by, SUM(b.price * p.quantity) AS revenue
FROM bagel b, purchase p
WHERE b.made_by = p.franchise AND b.name = p.bagel_name
GROUP BY b.made_by;

Done.


made_by,revenue
BAGEL CORP,12.87
Bobs Bagels,85.98
eBagel,272.63


매우 흥미로운 결과이다!

끝으로 분석을 마무리 지으면서, 제품 별 순이익의 분포를 알아보고 싶다. *하지만* 좀 더 큰 그림을 그리기 위해서는 12개 이상의 제품을 판매하는 판매자들의 결과들만 보기 원한다.

이 문제의 해답을 얻기 위해 `HAVING` 절을 소개 한다. 이 절은 *집계*에 조건을 걸 수 있다.

In [31]:
%%sql
SELECT b.name, SUM(b.price * p.quantity) AS sales
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING SUM(p.quantity) > 12;

Done.


name,sales
Plain with shmear,84.51
eBagel Expansion Pack,272.63


마지막으로 실행한 질의문을 나눠서 어떤 식으로 처리되는지 단계별로 살펴보자:

먼저, FROM-WHERE 부분이 평가 된다:

In [32]:
%%sql
SELECT *
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise;

Done.


name,price,made_by,bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,1.99,Bobs Bagels,Plain with shmear,Bobs Bagels,1,12,28
Plain with shmear,1.99,Bobs Bagels,Plain with shmear,Bobs Bagels,4,24,None
Egg with shmear,2.39,Bobs Bagels,Egg with shmear,Bobs Bagels,2,6,47
eBagel Expansion Pack,1.99,eBagel,eBagel Expansion Pack,eBagel,1,137,5
Plain with shmear,0.99,BAGEL CORP,Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,0.99,BAGEL CORP,Plain with shmear,BAGEL CORP,3,1,17


(*여담: WHERE 절에서 `AND b.made_by = p.franchise` 를 제외 했다면 어떤 결과를 얻었을까? 복잡한 질의문을 쪼개서 과정을 살펴보는 것은 디버깅하는 좋은 순서이다!*)

다음으로, GROUP-BY 가 적용된다:

In [33]:
%%sql
SELECT 
    b.name,
    GROUP_CONCAT(b.price, ',') AS prices,
    GROUP_CONCAT(b.made_by, ',') AS made_bys,
    bagel_name, 
    GROUP_CONCAT(p.franchise, ',') AS franchises,
    GROUP_CONCAT(p.date, ',') AS dates,
    GROUP_CONCAT(p.quantity, ',') AS quantities,
    GROUP_CONCAT(p.purchaser_age, ',') AS purchaser_ages
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name;

Done.


name,prices,made_bys,bagel_name,franchises,dates,quantities,purchaser_ages
Egg with shmear,2.39,Bobs Bagels,Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,"1.99,1.99,0.99,0.99","Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP",Plain with shmear,"Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP","1,4,2,3","12,24,12,1","28,24,17"
eBagel Expansion Pack,1.99,eBagel,eBagel Expansion Pack,eBagel,1,137,5


*`GROUP_CONCAT` 함수는 예쁘게 표시하기 위해 썼을 뿐이다.*

다음으로 `HAVING`  조건이 적용된다:

In [34]:
%%sql
SELECT 
    b.name,
    GROUP_CONCAT(b.price, ',') AS prices,
    GROUP_CONCAT(b.made_by, ',') AS made_bys,
    bagel_name, 
    GROUP_CONCAT(p.franchise, ',') AS franchises,
    GROUP_CONCAT(p.date, ',') AS dates,
    SUM(p.quantity) AS total_quantity,
    GROUP_CONCAT(p.purchaser_age, ',') AS purchaser_ages
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING SUM(p.quantity) > 12;

Done.


name,prices,made_bys,bagel_name,franchises,dates,total_quantity,purchaser_ages
Plain with shmear,"1.99,1.99,0.99,0.99","Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP",Plain with shmear,"Bobs Bagels,Bobs Bagels,BAGEL CORP,BAGEL CORP","1,4,2,3",49,"28,24,17"
eBagel Expansion Pack,1.99,eBagel,eBagel Expansion Pack,eBagel,1,137,5


최종적으로 select 절 (projection)의 집계 함수들과 다른 연산들이 적용된다:

In [35]:
%%sql
SELECT b.name, SUM(b.price * p.quantity) AS sales
FROM bagel b, purchase p
WHERE b.name = p.bagel_name AND b.made_by = p.franchise
GROUP BY b.name
HAVING SUM(p.quantity) > 12;

Done.


name,sales
Plain with shmear,84.51
eBagel Expansion Pack,272.63


한정자: existential 과 universal
------------------------------------

여러 인터뷰를 통해 알게된 사실은 마켓에서 성공하기 위해서 "바른(shmear)" 것이 필요하다고 한다. 이름에 "shmear" 들어 있는 제품이 메뉴에 있는지 알아보자.

In [12]:
%sql SELECT DISTINCT made_by FROM bagel WHERE name LIKE '%shmear%';

 * sqlite://
Done.


made_by
Bobs Bagels
BAGEL CORP


존재 여부를 검사하는 것을 보고 **existential** 한정자라 한다. 위에서 본 것처럼, SQL에서 쉽게 작성할 수 있다. **universal** 한정자(모든 r에 대해 C(r)의 형태)는 좀 더 어렵지만, 나름 간단하게 표현할 수 있다. 

예를 위해, 이름에 "shmear"가 포함된 모든 경쟁사의 제품을 찾아보자:

In [37]:
%%sql
SELECT DISTINCT made_by
FROM bagel
WHERE made_by NOT IN (
    SELECT made_by
    FROM bagel
    WHERE name NOT LIKE '%shmear%');

Done.


made_by
Bobs Bagels


SQL에서 NULL 값
-----------------

다음의 이상한 질의문을 살펴보자:

In [38]:
%sql SELECT * FROM purchase WHERE bagel_name LIKE '%shmear%';

Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17
Plain with shmear,Bobs Bagels,4,24,None


In [39]:
%%sql SELECT * FROM purchase 
WHERE bagel_name LIKE '%shmear%' 
  AND (purchaser_age >= 5 OR purchaser_age < 5);

Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17


`NULL` 값은 특별하게 취급되고 있음을 볼 수 있다. SQL에서는 불리안 값이 세 종류이다:`TRUE`, `FALSE`, 그리고 `UNKNOWN`.  `NULL` 값과 상수의 비교는 `UNKNOWN`을 되돌려 준다. 관계식은 조건이 `TRUE`일 때만 처리가 된다. 이를 다루기 위해 다음과 같이 특별한 조치를 취할 수 있다. 

In [40]:
%%sql SELECT * FROM purchase
WHERE bagel_name LIKE '%shmear%'
  AND (purchaser_age >= 5 OR purchaser_age < 5 
       OR purchaser_age IS NULL);

Done.


bagel_name,franchise,date,quantity,purchaser_age
Plain with shmear,Bobs Bagels,1,12,28
Egg with shmear,Bobs Bagels,2,6,47
Plain with shmear,BAGEL CORP,2,12,24
Plain with shmear,BAGEL CORP,3,1,17
Plain with shmear,Bobs Bagels,4,24,None


join 에 null 이 있으면 어떻게 되는가?

In [41]:
%%sql 
SELECT DISTINCT b.name 
FROM bagel b, purchase p 
WHERE b.name = p.bagel_name AND b.made_by = p.franchise;

Done.


name
Plain with shmear
Egg with shmear
eBagel Expansion Pack


구매된 적이 없는 베이글에 대한 정보를 잃게 된다!

Inner 와 Outer Joins
--------------------

바로 앞에서 살펴 본`WHERE` 절을 사용하는 join 질의문은 `INNER JOIN`이라고 부른다. 그리고 다음과 같이 다시 작성될 수 있다.

In [42]:
%%sql 
SELECT DISTINCT b.name 
FROM bagel b
    INNER JOIN purchase p ON b.name = p.bagel_name AND b.made_by = p.franchise;

Done.


name
Plain with shmear
Egg with shmear
eBagel Expansion Pack


테이블 `A`와 `B`를 join 조건 `C(A,B)`로 `INNER JOIN`한다면 `C(a,b) = TRUE`인 릴레이션 `(a,b)`을 돌려준다. 만약, 위에서 살펴 본 예와 같이 `C(a,b)`가 참인 `b`가 없다면, 결과에 `a`는 포함되지 않는다. 

대신 `OUTER JOIN` 을 쓸 수 있고, 세 종류가 있다: `LEFT`, `RIGHT`, 그리고 `FULL`. 

지금의 상황에서 우리가 필요한 것은 `LEFT OUTER JOIN` 이다.  Left outer join 은 왼쪽의 릴레이션 `a`에 대해 `C(a,b) = TRUE`를 만족하는 `b`가 없다하더라도 `(a, NULL)` 의 결과를 출력한다:

In [43]:
%%sql 
SELECT DISTINCT b.name 
FROM bagel b
    LEFT OUTER JOIN purchase p ON b.name = p.bagel_name AND b.made_by = p.franchise;

Done.


name
Plain with shmear
Egg with shmear
eBagel Drinkable Bagel
eBagel Expansion Pack
Organic Flax-seed bagel chips
